# פרויקט אמצע סמסטר - ניתוח נתונים
### מתן אלמליח 205625221 | דניאל פרץ

---

**הבעיה העסקית:** בחינת הקשר בין מאפייני הסרט (ז'אנר ודירוג איכות) לבין הצלחתו הכלכלית (Gross Revenue).

**מטרת האנליזה:** הדאטה-סט מכיל את 1,000 הסרטים המובילים בדירוג. המטרה שלנו היא לאפיין את האוכלוסייה הזו ולבדוק האם סרטים בעלי דירוג איכות גבוה יותר, או מז'אנרים מסוימים, נוטים להרוויח יותר בקופות.

**מהלך העבודה:**
* **ניקוי נתונים:** המרת נתונים טקסטואליים (כסף וזמן) למספרים וטיפול בערכים חסרים וחריגים.
* **ניתוח חד-משתני (Uni-variate):** מיפוי התפלגות הציונים והז'אנרים השכיחים.
* **ניתוח רב-משתני (בונוס):** הצלבת הנתונים לזיהוי קשר בין איכות, ז'אנר והכנסות.

In [ ]:
# Import librariesimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as sns

---
## סעיף 1: טעינת הנתונים
העלאת קובץ הנתונים לפייתון

In [ ]:
# Load data from CSV filedf = pd.read_csv('imdb_top_1000.csv')

---
## סעיף 2: הצגת רשומות
סקירה ראשונית של הנתונים

In [ ]:
# Display first 5 rowsdf.head()

In [ ]:
# Display data types and missing values infodf.info()

---
## סעיף 3: בדיקת וטיפול בכפילויות
זיהוי והסרת שורות כפולות

In [ ]:
# Check for duplicate rowsdup_count = df.duplicated().sum()print(f"Number of duplicate rows: {dup_count}")

In [ ]:
# Remove duplicate rowsdf.drop_duplicates(inplace=True)

---
## סעיף 4: שינוי ותיקון שמות עמודות
עדכון שמות עמודות לשמות ברורים יותר

In [ ]:
# Rename column to a clearer namedf.rename(columns={'Series_Title': 'Movie_Name'}, inplace=True)df.head(2)

---
## סעיף 5: טיפול בערכים לא תקינים
המרת נתונים טקסטואליים למספרים

In [ ]:
# Convert Runtime: remove 'min' and convert to integerdf['Runtime'] = df['Runtime'].astype(str).str.replace(' min', '').astype(int)# Convert Gross: remove commas (e.g. '1,000' -> '1000')df['Gross'] = df['Gross'].str.replace(',', '')# Convert to numeric - errors='coerce' turns invalid values to NaNdf['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')df.info()

---
## סעיף 6: טיפול בערכים חסרים
מילוי ערכים חסרים בשיטות מתאימות

In [ ]:
# Calculate median of Gross revenueGross_median = df['Gross'].median()# Fill missing values with mediandf['Gross'] = df['Gross'].fillna(Gross_median)# Verify no missing values remainprint(f"Missing values in Gross: {df['Gross'].isnull().sum()}")

In [ ]:
# Handle missing values in remaining columns# Meta_score: fill with medianMeta_median = df['Meta_score'].median()df['Meta_score'] = df['Meta_score'].fillna(Meta_median)# Certificate: fill with 'Unknown'df['Certificate'] = df['Certificate'].fillna('Unknown')# Verify final data statedf.info()

---
## סעיף 7: סיכום סטטיסטי של הנתונים
סטטיסטיקה תיאורית למשתנים הנומריים: ממוצע, סטיית תקן, מינימום, מקסימום, חציון ורבעונים

In [ ]:
# Display descriptive statisticsdf.describe()

---
## סעיף 8: זיהוי וטיפול בערכי קיצון (Outliers)
איתור ערכים חריגים בשיטת IQR וטיפול בהם

In [ ]:
# Detect outliers using IQR methodnumerical_cols = ['Gross', 'Runtime', 'Meta_score', 'No_of_Votes']def detect_outliers_iqr(df, column):    """Detect outliers using IQR method"""    Q1 = df[column].quantile(0.25)    Q3 = df[column].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]    return len(outliers), lower_bound, upper_boundprint('=== Outlier Detection using IQR ===')print()for col in numerical_cols:    count, lower, upper = detect_outliers_iqr(df, col)    print(f'{col}:')    print(f'  Lower bound: {lower:,.2f}')    print(f'  Upper bound: {upper:,.2f}')    print(f'  Number of outliers: {count}')    print()

In [ ]:
# Visualize outliers using Boxplotsfig, axes = plt.subplots(2, 2, figsize=(14, 10))sns.boxplot(data=df, y='Gross', ax=axes[0, 0], color='steelblue')axes[0, 0].set_title('Gross Revenue Distribution')axes[0, 0].set_ylabel('Gross ($)')sns.boxplot(data=df, y='Runtime', ax=axes[0, 1], color='coral')axes[0, 1].set_title('Runtime Distribution')axes[0, 1].set_ylabel('Minutes')sns.boxplot(data=df, y='Meta_score', ax=axes[1, 0], color='forestgreen')axes[1, 0].set_title('Meta Score Distribution')axes[1, 0].set_ylabel('Score')sns.boxplot(data=df, y='No_of_Votes', ax=axes[1, 1], color='orchid')axes[1, 1].set_title('Number of Votes Distribution')axes[1, 1].set_ylabel('Votes')plt.tight_layout()plt.suptitle('Outlier Detection using Boxplots', y=1.02, fontsize=14)plt.show()

In [ ]:
# Handle outliers using Capping methoddef cap_outliers(df, column):    """Cap outliers to IQR bounds"""    Q1 = df[column].quantile(0.25)    Q3 = df[column].quantile(0.75)    IQR = Q3 - Q1    lower_bound = Q1 - 1.5 * IQR    upper_bound = Q3 + 1.5 * IQR    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)    return df# Apply capping to Gross and No_of_Votesprint('=== Outlier Treatment ===')print('Applying Capping to Gross and No_of_Votes')print()print('Before treatment:')print(f"Gross - Max: {df['Gross'].max():,.0f}, Min: {df['Gross'].min():,.0f}")print(f"No_of_Votes - Max: {df['No_of_Votes'].max():,}, Min: {df['No_of_Votes'].min():,}")df = cap_outliers(df, 'Gross')df = cap_outliers(df, 'No_of_Votes')print()print('After treatment:')print(f"Gross - Max: {df['Gross'].max():,.0f}, Min: {df['Gross'].min():,.0f}")print(f"No_of_Votes - Max: {df['No_of_Votes'].max():,}, Min: {df['No_of_Votes'].min():,}")

In [ ]:
# Boxplots after outlier treatmentfig, axes = plt.subplots(1, 2, figsize=(12, 5))sns.boxplot(data=df, y='Gross', ax=axes[0], color='steelblue')axes[0].set_title('Gross Revenue (After Capping)')axes[0].set_ylabel('Gross ($)')sns.boxplot(data=df, y='No_of_Votes', ax=axes[1], color='orchid')axes[1].set_title('Number of Votes (After Capping)')axes[1].set_ylabel('Votes')plt.tight_layout()plt.suptitle('Boxplots After Outlier Treatment', y=1.02, fontsize=14)plt.show()

---
## סעיף 9: ניתוח חד-משתני (Univariate Analysis)
ניתוח והצגה של כל משתנה בנפרד באמצעות היסטוגרמות וגרפי עמודות

In [ ]:
# Histograms for numerical variablesfig, axes = plt.subplots(2, 2, figsize=(14, 10))sns.histplot(df['Gross'], bins=20, kde=True, ax=axes[0, 0], color='steelblue')axes[0, 0].set_title('Gross Revenue Distribution')axes[0, 0].set_xlabel('Gross ($)')axes[0, 0].set_ylabel('Count')sns.histplot(df['Runtime'], bins=20, kde=True, ax=axes[0, 1], color='coral')axes[0, 1].set_title('Runtime Distribution')axes[0, 1].set_xlabel('Minutes')axes[0, 1].set_ylabel('Count')sns.histplot(df['Meta_score'], bins=20, kde=True, ax=axes[1, 0], color='forestgreen')axes[1, 0].set_title('Meta Score Distribution')axes[1, 0].set_xlabel('Score')axes[1, 0].set_ylabel('Count')sns.histplot(df['No_of_Votes'], bins=20, kde=True, ax=axes[1, 1], color='orchid')axes[1, 1].set_title('Number of Votes Distribution')axes[1, 1].set_xlabel('Votes')axes[1, 1].set_ylabel('Count')plt.tight_layout()plt.suptitle('Distribution of Numerical Variables', y=1.02, fontsize=14)plt.show()

In [ ]:
# IMDB Rating distributionplt.figure(figsize=(10, 5))sns.histplot(df['IMDB_Rating'], bins=15, kde=True, color='gold')plt.title('IMDB Rating Distribution')plt.xlabel('Rating')plt.ylabel('Count')plt.show()

In [ ]:
# Movies distribution by decadedf['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')df['Decade'] = (df['Released_Year'] // 10) * 10plt.figure(figsize=(12, 6))decade_counts = df['Decade'].value_counts().sort_index()sns.barplot(x=decade_counts.index.astype(int), y=decade_counts.values, palette='viridis')plt.title('Movies Distribution by Decade')plt.xlabel('Decade')plt.ylabel('Number of Movies')plt.xticks(rotation=45)plt.show()

In [ ]:
# Genre distributiondf['main_Genre'] = df['Genre'].astype(str).str.split(',').str[0]plt.figure(figsize=(12, 6))sns.countplot(    data=df,    x='main_Genre',    order=df['main_Genre'].value_counts().index[:10],    palette='Set2')plt.title('Top 10 Most Common Genres')plt.xlabel('Genre')plt.ylabel('Number of Movies')plt.xticks(rotation=45)plt.show()

---
## בונוס: ניתוח רב-משתני (Multivariate Analysis)
בחינת הקשר בין דירוג IMDB, הכנסות וז'אנר באמצעות גרף פיזור

In [ ]:
# Scatter plot: Rating vs Revenue vs Genredf['Gross_millions'] = df['Gross'] / 1000000plt.figure(figsize=(12, 8))sns.scatterplot(    data=df,    x='IMDB_Rating',    y='Gross_millions',    hue='main_Genre',    s=100,    alpha=0.7)plt.title('IMDB Rating vs Gross Revenue by Genre')plt.xlabel('IMDB Rating')plt.ylabel('Gross Revenue (Millions $)')plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left')plt.tight_layout()plt.show()